# Stacking

Julian Domingo - jad5348

In [1]:
# Data analysis 
import pandas as pd
import numpy as np

# Modeling stuff
from xgboost import XGBClassifier
from mlens.ensemble import SuperLearner
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (RandomForestClassifier, 
                              AdaBoostClassifier,
                              ExtraTreesClassifier,
                              BaggingClassifier)
from sklearn.model_selection import GridSearchCV, cross_val_score

# Computation / numerical
from scipy.stats import uniform, randint
from sklearn.metrics import roc_auc_score
from mlens.preprocessing import Subset
from mlens.metrics import make_scorer
from mlens.model_selection import Evaluator

# For reproducibility
seed = 42
np.random.seed(seed)

# Constants
n_splits = 5

[MLENS] backend: threading


In [2]:
test_ids = pd.read_csv("./data/raw/test.csv")[["id"]]
y_train = pd.read_csv("./data/raw/train.csv")["Y"].ravel()

#### Helper Functions

In [3]:
def save_preds(preds, preds_filename):
    submission = pd.DataFrame({"id": test_ids.id, "Y": preds})
    submission.to_csv("./submissions/stacking_{}.csv".format(preds_filename), index=False, columns=["id", "Y"])
    
    
def get_data(filename):
    train = pd.read_csv("./data/refined/train/train_{}.csv".format(filename))
    test = pd.read_csv("./data/refined/test/test_{}.csv".format(filename))
    
    x_train = train.drop(["Y"], axis = 1)
    y_train = train["Y"]
    
    return train, test, x_train, y_train


def get_cross_val_score(model, x_train, y_train, n_folds, run_parallel=True):
    if run_parallel:
        cv = cross_val_score(model, x_train, y_train, cv = n_folds, scoring = "roc_auc", n_jobs = -1)
    else:
        cv = cross_val_score(model, x_train, y_train, cv = n_folds, scoring = "roc_auc")

    print("Cross validation score: {} +/- {}\nRaw scores: {}".format(str(np.mean(cv)), str(np.std(cv)), str(cv)))
    return cv

In [4]:
train_raw, test_raw, x_train_raw, y_train_raw = get_data("raw")
train_base, test_base, x_train_base, y_train_base = get_data("base")
train_log, test_log, x_train_log, y_train_log = get_data("log")
train_poly, test_poly, x_train_poly, y_train_poly = get_data("poly")
train_scaled, test_scaled, x_train_scaled, y_train_scaled = get_data("scaled")

#### Stacking Class

Stacking works well for small to medium-sized data sets.

In [5]:
class Stacker(object):
    def __init__(self, base_learners, meta_learners, y_train, test_ids):
        self.base_learners = base_learners
        self.meta_learners = meta_learners
        self.y_train = y_train
        self.test_ids = test_ids
    
    
    def get_indiv_meta_preds(self, meta):
        """ Retrieves the predictions of the model 'meta'. """
        if self.meta_features_train is None or self.meta_features_test is None:
            raise ValueError("Invoke 'get_meta_features' before predicting.")
            
        meta.fit(self.meta_features_train, self.y_train)
        meta_preds = meta.predict_proba(self.meta_features_test)[:,1]
        return meta_preds
        
        
    def get_meta_features(self):
        """ Retrieves all meta features for the train & test data from the base learners specified. """
        self.meta_features_train = np.zeros((len(self.y_train), len(self.base_learners)))
        self.meta_features_test = np.zeros((len(self.test_ids), len(self.base_learners)))
        
        for i, base in enumerate(self.base_learners):
            print ("Gathering meta feature from '{}'...".format(base))
            self.meta_features_train[:, i] = pd.read_csv("./meta_features/train/train_{}.csv".format(base), \
                                                         index_col=0).as_matrix().ravel()
            self.meta_features_test[:, i] = pd.read_csv("./meta_features/test/test_{}.csv".format(base), \
                                                        index_col=0).as_matrix().ravel()
            
        return self.meta_features_train.copy(), self.meta_features_test.copy()
    
    
    def fit_meta_learners_and_predict(self):
        """ Generates predictions using all meta features generated from the base learners for each meta learner. """
        if not self.meta_features_train or self.meta_features_test:
            raise ValueError("get_meta_features() should be called before generate_out_of_folds_preds.")
        
        self.meta_learner_preds = np.zeros(len(self.test_ids), len(self.meta_learners))
        
        for i, meta in enumerate(meta_learners):
            meta.fit(self.meta_features_train, self.y_train)
            self.meta_learner_preds[:, i] =  meta.predict_proba(self.meta_features_test)[:,1]
            
    
    def get_df_meta_learner_preds(self):
        if self.meta_learner_preds is None:
            raise ValueError("No predictions were found. Invoke 'fit_meta_learners_and_predict' first.")
        
        if self.base_learners is None:
            raise ValueError("No base learners were specified. Construct an instance with base learners.")
        
        return pd.DataFrame(self.meta_learner_preds, columns=self.base_learners)
    
    
    def get_final_preds(self):
        return np.mean(self.meta_learner_preds, axis=1)

## Meta Learner(s) Parameter Tuning & Predictions

### Ensemble 1
**Base Learners: **
    * RF Raw
    * RF Log
    * RF Poly
    * XGB Raw
    * XGB Base
    * XGB Poly
    * LR Log
    * Ada Base
   
**Meta Learners: **
    * RF

In [31]:
base_learners_v1 = [
    "random_forest_raw",
    "random_forest_log",
    "random_forest_poly",
    "xgboost_raw",
    "xgboost_base",
    "xgboost_poly",
    "logistic_regression_log",
    "adaboost_base"
]

rf_meta_v1 = RandomForestClassifier(n_jobs=-1)

meta_learners_v1 = [rf_meta_v1]

In [32]:
stacker_v1 = Stacker(base_learners_v1, meta_learners_v1, y_train, test_ids)
meta_features_train_v1, meta_features_test_v1 = stacker_v1.get_meta_features()

Gathering meta feature from 'random_forest_raw'...
Gathering meta feature from 'random_forest_log'...
Gathering meta feature from 'random_forest_poly'...
Gathering meta feature from 'xgboost_raw'...
Gathering meta feature from 'xgboost_base'...
Gathering meta feature from 'xgboost_poly'...
Gathering meta feature from 'logistic_regression_log'...
Gathering meta feature from 'adaboost_base'...


In [11]:
# Warning: this block takes a long time to execute.
rf_meta_v1_param_grid = {
    "max_features": range(5, 7 + 1),
    "max_depth": range(5, 10 + 1),
    "n_estimators": range(300, 1000 + 1, 100) 
}

gs_rf_v1 = GridSearchCV(estimator=rf_meta_v1, param_grid=rf_meta_v1_param_grid, cv=n_splits, scoring="roc_auc", n_jobs=-1)
gs_rf_v1.fit(meta_features_train_v1, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_features': [5, 6, 7], 'n_estimators': [300, 400, 500, 600, 700, 800, 900, 1000], 'max_depth': [5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [10]:
print gs_rf_v1.best_params_
print gs_rf_v1.best_score_

{'max_features': 6, 'n_estimators': 300, 'max_depth': 6}
0.772539082175


In [33]:
print get_cross_val_score(RandomForestClassifier(n_jobs=-1, n_estimators=300, max_depth=6, max_features=6), meta_features_train_v1, y_train, n_folds=5)

Cross validation score: 0.771359868469 +/- 0.00723717612684
Raw scores: [ 0.76715711  0.77956341  0.75947714  0.77377431  0.77682737]
[ 0.76715711  0.77956341  0.75947714  0.77377431  0.77682737]


In [19]:
# Get predictions w/ tuned params
meta_preds_v1 = stacker_v1.get_indiv_meta_preds(RandomForestClassifier(n_jobs=-1, n_estimators=300, max_features=6, max_depth=6))
print meta_preds_v1

[ 0.97136003  0.67447269  0.94252603 ...,  0.88289829  0.98178302
  0.96819782]


In [25]:
# Get predictions w/o tuned params
meta_preds_v1_untuned = stacker_v1.get_indiv_meta_preds(RandomForestClassifier(n_jobs=-1))
print meta_preds_v1_untuned

[ 1.   0.7  0.9 ...,  1.   1.   1. ]


As seen by the predictions without tuned parameters, tuning of the meta model makes a tremendous difference on the prediction set obtained.

### Ensemble 2
**Base Learners: **
    * RF Raw
    * RF Log
    * RF Poly
    * XGB Raw
    * XGB Base
    * XGB Poly
    * LR Log
    * Ada Base
    * KNN_{2, 4, 8, 16, 32, 64, 128, 256, 512, 1024}
   
**Meta Learners: **
    * RF

In [24]:
base_learners_v2 = [
    "random_forest_raw",
    "random_forest_log",
    "random_forest_poly",
    "xgboost_raw",
    "xgboost_base",
    "xgboost_poly",
    "logistic_regression_log",
    "adaboost_base",
    "knn_2",
    "knn_4",
    "knn_8",
    "knn_16",
    "knn_32",
    "knn_64",
    "knn_128",
    "knn_256",
    "knn_512",
    "knn_1024"
]

rf_meta_v2 = RandomForestClassifier(n_jobs=-1)
meta_learners_v2 = [rf_meta_v2]

stacker_v2 = Stacker(base_learners_v2, meta_learners_v2, y_train, test_ids)
meta_features_train_v2, meta_features_test_v2 = stacker_v2.get_meta_features()

Gathering meta feature from 'random_forest_raw'...
Gathering meta feature from 'random_forest_log'...
Gathering meta feature from 'random_forest_poly'...
Gathering meta feature from 'xgboost_raw'...
Gathering meta feature from 'xgboost_base'...
Gathering meta feature from 'xgboost_poly'...
Gathering meta feature from 'logistic_regression_log'...
Gathering meta feature from 'adaboost_base'...
Gathering meta feature from 'knn_2'...
Gathering meta feature from 'knn_4'...
Gathering meta feature from 'knn_8'...
Gathering meta feature from 'knn_16'...
Gathering meta feature from 'knn_32'...
Gathering meta feature from 'knn_64'...
Gathering meta feature from 'knn_128'...
Gathering meta feature from 'knn_256'...
Gathering meta feature from 'knn_512'...
Gathering meta feature from 'knn_1024'...


In [29]:
# Warning: this block takes a long time to execute.
rf_meta_v2_param_grid = {
    "max_features": range(5, 7 + 1),
    "max_depth": range(5, 10 + 1),
    "n_estimators": range(300, 1000 + 1, 100) 
}

gs_rf_v2 = GridSearchCV(estimator=rf_meta_v2, param_grid=rf_meta_v2_param_grid, cv=n_splits, scoring="roc_auc", n_jobs=-1)
gs_rf_v2.fit(meta_features_train_v2, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_features': [5, 6, 7], 'n_estimators': [300, 400, 500, 600, 700, 800, 900, 1000], 'max_depth': [5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [30]:
print gs_rf_v2.best_params_
print gs_rf_v2.best_score_

{'max_features': 5, 'n_estimators': 300, 'max_depth': 9}
0.773074767924


In [30]:
print get_cross_val_score(RandomForestClassifier(n_jobs=-1, n_estimators=300, max_depth=9, max_features=5), meta_features_train_v2, y_train, n_folds=5)

Cross validation score: 0.769897637451 +/- 0.00448583181217
Raw scores: [ 0.76837689  0.77614972  0.77087385  0.76246286  0.77162487]
[ 0.76837689  0.77614972  0.77087385  0.76246286  0.77162487]


In [31]:
# Get predictions w/ tuned params
meta_preds_v2_tuned = stacker_v2.get_indiv_meta_preds(RandomForestClassifier(n_jobs=-1, n_estimators=300, max_depth=9, max_features=5))
print meta_preds_v2_tuned

[ 0.97460188  0.66699757  0.95640802 ...,  0.91079086  0.98644606
  0.97122791]


In [33]:
save_preds(meta_preds_v2_tuned, "_".join(base_learners_v2))

### Ensemble 3
**Base Learners: **
    * RF Raw
    * RF Log
    * RF Poly
    * XGB Raw
    * XGB Base
    * XGB Poly
    * LR Log
    * Ada Base
    * Bagged XGB (50 runs)
    * Extra Trees Base
    
**Meta Learners: **
    * RF
    * XGB
    
**Final Predictions**: harmonic mean of meta learner predictions

In [6]:
base_learners_v3 = [
    "random_forest_raw",
    "random_forest_log",
    "random_forest_poly",
    "xgboost_raw",
    "xgboost_base",
    "xgboost_poly",
    "xgboost_base",
    "xgboost_bag",
    "logistic_regression_log",
    "adaboost_base",
    "extra_trees_base"
]

rf_meta_v3 = RandomForestClassifier(n_jobs=-1)
meta_learners_v3 = [rf_meta_v3]

stacker_v3 = Stacker(base_learners_v3, meta_learners_v3, y_train, test_ids)
meta_features_train_v3, meta_features_test_v3 = stacker_v3.get_meta_features()

Gathering meta feature from 'random_forest_raw'...
Gathering meta feature from 'random_forest_log'...
Gathering meta feature from 'random_forest_poly'...
Gathering meta feature from 'xgboost_raw'...
Gathering meta feature from 'xgboost_base'...
Gathering meta feature from 'xgboost_poly'...
Gathering meta feature from 'xgboost_base'...
Gathering meta feature from 'xgboost_bag'...
Gathering meta feature from 'logistic_regression_log'...
Gathering meta feature from 'adaboost_base'...
Gathering meta feature from 'extra_trees_base'...


In [7]:
#meta_learners = [RandomForestClassifier(), XGBClassifier()]
meta_learners = [RandomForestClassifier()]

v3_param_grid = {
#     "xgb":
#     {
#         "learning_rate": uniform(0.01, 0.2),
#         "n_estimators": randint(300, 600),
#         "max_depth": randint(3, 10),
#         "min_child_weight": uniform(0.5, 1.5),
#         "gamma": uniform(0, 0.2),
#         "subsample": uniform(0.5, 0.9),
#         "colsample_bytree": uniform(0.5, 0.9),
#     },
    "rf":
    {
        "max_features": randint(3, 7),
        "max_depth": randint(5, 10),
        "n_estimators": randint(300, 600)
    }
}

base_learners_v3_models = [
    RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=1000, n_jobs=-1),
    LogisticRegression(C=100, tol=1e-05, solver="liblinear"),
    ExtraTreesClassifier(max_depth=12, n_estimators=250, n_jobs=-1, criterion="entropy"),
    AdaBoostClassifier(n_estimators=395, learning_rate=1.55),
    XGBClassifier(learning_rate=0.1,
                    n_estimators=345,
                    max_depth=8,
                    min_child_weight=2,
                    gamma=0.2,
                    subsample=0.6,
                    colsample_bytree=0.5,
                    objective="binary:logistic",
                    n_jobs=-1,
                    scale_pos_weight=1,
                    seed=seed),
    BaggingClassifier(base_estimator=XGBClassifier(learning_rate=0.1,
                                                    n_estimators=345,
                                                    max_depth=8,
                                                    min_child_weight=2,
                                                    gamma=0.2,
                                                    subsample=0.6,
                                                    colsample_bytree=0.5,
                                                    objective="binary:logistic",
                                                    n_jobs=-1,
                                                    scale_pos_weight=1,
                                                    seed=seed), 
                      n_estimators=50, max_samples=0.7, max_features=0.75, bootstrap_features=True)
]

Running an entire ensemble several times just to compare different meta learners can be prohibitvely expensive. ML-Ensemble implements a class that acts as a transformer, allowing you to use ingoing layers as a "preprocessing" step, so that you need to only evaluate the meta learners.

In [9]:
# Put the layers you don't want to tune into an ensemble with model selection turned on
in_layer = SuperLearner(model_selection=True)
in_layer.add(base_learners_v3_models)

preprocess = [in_layer]

In [ ]:
evl = Evaluator(
    make_scorer(roc_auc_score, greater_is_better=True, needs_proba=True),
    cv=n_splits,
    random_state=seed,
    verbose=5,
)

evl.fit(
    meta_features_train_v3, 
    y_train,
    meta_learners,
    v3_param_grid,
    preprocessing={"meta": preprocess},
    n_iter=4                            # bump this up to do a larger grid search
)

/Users/julian/anaconda2/lib/python2.7/site-packages/mlens/model_selection/model_selection.py:600: UserWarning: No valid parameters found for meta.randomforestclassifier. Will fit and score once with given parameter settings.
  "settings.".format(key))


Launching job
Preprocessing 1 preprocessing pipelines over 5 CV folds


/Users/julian/anaconda2/lib/python2.7/site-packages/xgboost-0.7-py2.7.egg/xgboost/sklearn.py:200: DeprecationWarning: The seed parameter is deprecated as of version .6.Please use random_state instead.seed is deprecated.
  'seed is deprecated.', DeprecationWarning)


In [ ]:
pd.DataFrame(evl.results)

### ML-ensemble Predictions

#### v1

In [77]:
rf_meta_mlens_v1 = RandomForestClassifier(n_jobs=-1, n_estimators=300, max_depth=6)

mlens_base_learners = [
    RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=1000, n_jobs=-1),
    LogisticRegression(C=100, tol=1e-05, solver="liblinear"),
    AdaBoostClassifier(n_estimators=395, learning_rate=1.55),
    XGBClassifier(learning_rate=0.1,
                    n_estimators=345,
                    max_depth=8,
                    min_child_weight=2,
                    gamma=0.2,
                    subsample=0.6,
                    colsample_bytree=0.5,
                    objective="binary:logistic",
                    n_jobs=-1,
                    scale_pos_weight=1,
                    seed=seed)
]

ensemble_v1 = SuperLearner(random_state=seed, scorer=roc_auc_score)
ensemble_v1.add(mlens_base_learners, proba=True)
ensemble_v1.add_meta(rf_meta_mlens_v1, proba=True)
mlens_v1_preds = ensemble_v1.fit(meta_features_train_v1, y_train).predict_proba(meta_features_test_v1)[:,1]

In [78]:
print mlens_v1_preds
save_preds(mlens_v1_preds, "mlens_v1")

[ 0.9640277   0.70394427  0.95102447 ...,  0.89943802  0.98440754
  0.95957553]


#### v2

In [34]:
rf_meta_mlens_v2 = RandomForestClassifier(n_jobs=-1, n_estimators=300, max_depth=9, max_features=5)

mlens_base_learners = [
    RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=1000, n_jobs=-1),
    LogisticRegression(C=100, tol=1e-05, solver="liblinear"),
    ExtraTreesClassifier(max_depth=12, n_estimators=250, n_jobs=-1, criterion="entropy"),
    AdaBoostClassifier(n_estimators=395, learning_rate=1.55),
    XGBClassifier(learning_rate=0.1,
                    n_estimators=345,
                    max_depth=8,
                    min_child_weight=2,
                    gamma=0.2,
                    subsample=0.6,
                    colsample_bytree=0.5,
                    objective="binary:logistic",
                    n_jobs=-1,
                    scale_pos_weight=1,
                    seed=seed),
    KNeighborsClassifier(n_jobs=-1, n_neighbors=2),
    KNeighborsClassifier(n_jobs=-1, n_neighbors=4),
    KNeighborsClassifier(n_jobs=-1, n_neighbors=8),
    KNeighborsClassifier(n_jobs=-1, n_neighbors=16),
    KNeighborsClassifier(n_jobs=-1, n_neighbors=32),
    KNeighborsClassifier(n_jobs=-1, n_neighbors=64),
    KNeighborsClassifier(n_jobs=-1, n_neighbors=128),
    KNeighborsClassifier(n_jobs=-1, n_neighbors=256),
    KNeighborsClassifier(n_jobs=-1, n_neighbors=512),
    KNeighborsClassifier(n_jobs=-1, n_neighbors=1024)
]

ensemble_v2 = SuperLearner(random_state=seed, scorer=roc_auc_score)
ensemble_v2.add(mlens_base_learners, proba=True)
ensemble_v2.add_meta(rf_meta_mlens_v2, proba=True)
mlens_v2_preds = ensemble_v2.fit(meta_features_train_v2, y_train).predict_proba(meta_features_test_v2)[:,1]

In [35]:
print mlens_v2_preds
save_preds(mlens_v2_preds, "mlens_v2")

[ 0.97149235  0.76364833  0.96501911 ...,  0.92276353  0.98632556
  0.96678698]


After looking at the LB scores, it seems adding KNeighborsClassifier models don't improve the stacking ensemble.

#### v3

In [42]:
rf_meta_mlens_v3 = RandomForestClassifier(n_jobs=-1, n_estimators=300, max_depth=9, max_features=5)

mlens_base_learners_v3 = [
    RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=1000, n_jobs=-1),
    LogisticRegression(C=100, tol=1e-05, solver="liblinear"),
    ExtraTreesClassifier(max_depth=12, n_estimators=250, n_jobs=-1, criterion="entropy"),
    AdaBoostClassifier(n_estimators=395, learning_rate=1.55),
    XGBClassifier(learning_rate=0.1,
                    n_estimators=345,
                    max_depth=8,
                    min_child_weight=2,
                    gamma=0.2,
                    subsample=0.6,
                    colsample_bytree=0.5,
                    objective="binary:logistic",
                    n_jobs=-1,
                    scale_pos_weight=1,
                    seed=seed),
    BaggingClassifier(base_estimator=XGBClassifier(learning_rate=0.1,
                                                    n_estimators=345,
                                                    max_depth=8,
                                                    min_child_weight=2,
                                                    gamma=0.2,
                                                    subsample=0.6,
                                                    colsample_bytree=0.5,
                                                    objective="binary:logistic",
                                                    n_jobs=-1,
                                                    scale_pos_weight=1,
                                                    seed=seed), 
                      n_estimators=50, max_samples=0.7, max_features=0.75, bootstrap_features=True)
]

ensemble_v3 = SuperLearner(random_state=seed, scorer=roc_auc_score)
ensemble_v3.add(mlens_base_learners_v3, proba=True)
ensemble_v3.add_meta(rf_meta_mlens_v3, proba=True)
mlens_v3_preds = ensemble_v3.fit(meta_features_train_v3, y_train).predict_proba(meta_features_test_v3)[:,1]

,random_forest_raw,random_forest_log,random_forest_poly,xgboost_raw,xgboost_base,xgboost_poly,xgboost_base,xgboost_bag,logistic_regression_log,adaboost_base,extra_trees_base
0,0.871,0.902,0.831,0.938816,0.894083,0.682318,0.894083,0.958313,0.912384,0.501568,0.908163
1,0.971,0.973,0.977,0.997241,0.994538,0.994200,0.994538,0.992380,0.935845,0.501776,0.953350
2,0.982,0.982,0.976,0.994968,0.991531,0.997939,0.991531,0.980984,0.950247,0.501754,0.939202
3,0.978,0.989,0.957,0.999398,0.998098,0.999019,0.998098,0.994178,0.966773,0.502810,0.969778
4,0.958,0.933,0.941,0.951995,0.964378,0.965681,0.964378,0.922798,0.937737,0.500819,0.928275
5,0.883,0.898,0.888,0.973064,0.960177,0.982531,0.960177,0.954669,0.937885,0.500930,0.948550
6,0.905,0.922,0.896,0.926855,0.918181,0.970848,0.918181,0.942915,0.935959,0.502308,0.932186
7,0.996,0.996,0.986,0.985542,0.992321,0.989989,0.992321,0.983988,0.945346,0.501293,0.937382
8,0.979,0.972,0.987,0.996340,0.994799,0.993424,0.994799,0.964199,0.942417,0.501341,0.941501
9,0.984,0.985,0.981,0.997901,0.995806,0.996160,0.995806,0.971569,0.938500,0.503214,0.940962
